In [1]:
from qcodes_contrib_drivers.drivers.Valon.Valon_5015 import Valon5015
import time
import numpy as np
from tqdm import trange, tqdm

In [2]:
addr = 'ASRL/dev/ttyUSB0::INSTR'
valon = Valon5015(name="Valon", address=addr)
valon.frequency(1452e6)
valon.offset(0)
valon.power(0)
valon.modulation_db(0)
valon.modulation_frequency(1)
valon.low_power_mode_enabled(True)
valon.buffer_amplifiers_enabled(False)
status = valon.status()
print(status)

-->amf 1
AMF 0.001 kHz
-->pdn 1
PDN 1;
-->oen 0
OEN 0;
-->stat
Valon Technology, 5019, 12202687, R5  version 1.2c   Build: Aug  8 2022  01:10:47
      VBAT = 1363   6.588 V
      UPTS = 1730    21.0 C
   PA TEMP = 1063    23.3 C
       +5V = 3061   4.932 V
       -5V = 1491  -5.116 V


In [76]:
ON = 0
valon.buffer_amplifiers_enabled(bool(ON))

In [7]:
Pin = 3
valon.power(Pin)

In [8]:
def V_to_P(V): 
    P = 10*np.log10(V**2/100*1000) 
    return P

def P_to_V(P): 
    V = np.sqrt(10**(P/10)/1000*100) 
    return V

In [30]:
V_to_P(0.25), P_to_V(-2), V_to_P(0.63), P_to_V(6)

(np.float64(-2.041199826559248),
 np.float64(0.251188643150958),
 np.float64(5.986810989071634),
 np.float64(0.6309573444801932))

In [74]:
temp = r'/home/electron/artiq/experiment/artiq-master/.temp/RF_temp.npz' 
file = np.load(temp) 
t_wait = file['t_wait']
t_load = file['t_load']
t_acquisition = file['t_acquisition']
N_average = file['N_average']
cyle_time = 10*N_average*(t_load + t_wait + t_acquisition)*1e-6
scan_min = file['RF_Vs_start'] 
scan_max = file['RF_Vs_stop'] 
scan_stp = file['RF_Vs_step']

In [75]:
# scan_min = 0.25 
# scan_max = 0.63 
# scan_stp = 0.005
# sweep_time = 1e-3
# N_average = 100

bar = tqdm(np.arange(scan_min, scan_max, scan_stp))
var = {'RF Power (V)': scan_min, 'RF Power (dBm)': V_to_P(scan_min)}
for V in bar:
    P = V_to_P(V)
    # valon.power(P)
    var['RF Power (V)'] = V
    var['RF Power (dBm)'] = P
    bar.set_postfix(var)
    time.sleep(cyle_time)

100%|██| 51/51 [11:54<00:00, 14.01s/it, RF Power (V)=0.435, RF Power (dBm)=2.77]
